# In-Context Learning


In-context learning is a generalisation of few-shot learning where the LLM is provided a context as part of the prompt and asked to respond by utilising the information in the context.

* Example: *"Summarize this research article into one paragraph highlighting its strengths and weaknesses: [insert article text]”*
* Example: *"Extract all the quotes from this text and organize them in alphabetical order: [insert text]”*

A very popular technique that you will learn in week 5 called Retrieval-Augmented Generation (RAG) is a form of in-context learning, where:
* a search engine is used to retrieve some relevant information
* that information is then provided to the LLM as context


In [ ]:
!pip install tqdm

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 5.3 MB/s eta 0:00:00


In this example we download some recent research papers from arXiv papers, extract the text from the PDF files and ask Gemini to summarize the articles as well as provide the main strengths and weaknesses of the papers. Finally we print the summaries to a local html file and as markdown.

In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from urllib.request import urlopen, urlretrieve
from IPython.display import Markdown, display
from pypdf import PdfReader
from datetime import date
from tqdm import tqdm

In [ ]:
API_KEY = " "
genai.configure(api_key=API_KEY)

We select those papers that have been featured in Hugging Face papers.

In [ ]:
BASE_URL = "https://huggingface.co/papers"
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.content, "html.parser")
h3s = soup.find_all("h3")

papers = []

for h3 in h3s:
    a = h3.find("a")
    title = a.text
    link = a["href"].replace('/papers', '')

    papers.append({"title": title, "url": f"https://arxiv.org/pdf{link}"})

Code to extract text from PDFs.

In [ ]:
def extract_paper(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text


def extract_pdf(url):
    pdf = urlretrieve(url, "pdf_file.pdf")
    reader = PdfReader("pdf_file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


def printmd(string):
    display(Markdown(string))

In [ ]:
LLM = "gemini-2.5-flash" #1.5 not availiable, 2.5 waiting for too long
model = genai.GenerativeModel(LLM)

We use Gemini to summarize the papers.

In [ ]:
for paper in tqdm(papers):
    try:
        paper["summary"] = model.generate_content("""Summarize this research article into one paragraph without formatting highlighting its strengths and weaknesses. Modify your output so that it is in the format of html table, there are two columns, one is strength and anoter one is weaknesss. You should present the text in a table, which means you should give me the result in such style:
    <table>
  <thead>
    <tr>
      <th>Strength</th>
      <th>Weakness</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Your strength of the paper</td>
      <td>Your weakness of the paper.</td>
    </tr>
  </tbody>
</table>
        """ + extract_pdf(paper["url"])).text
    except:
        print("Generation failed")
        paper["summary"] = "Paper not available"

  6%|▌         | 1/17 [00:02<00:42,  2.63s/it]

Generation failed


 12%|█▏        | 2/17 [00:07<00:55,  3.70s/it]

Generation failed


 18%|█▊        | 3/17 [00:15<01:22,  5.92s/it]

Generation failed


 24%|██▎       | 4/17 [00:17<00:53,  4.12s/it]

Generation failed


We print the results to a html file.

In [ ]:
page = f"<html> <head> <h1>Daily Dose of AI Research</h1> <h4>{date.today()}</h4> <p><i>Summaries generated with: {LLM}</i>"
with open("papers1.html", "w") as f:
    f.write(page)
for paper in papers:
    page = f'<h2><a href="{paper["url"]}">{paper["title"]}</a></h2> <p>{paper["summary"]}</p>'
    with open("papers.html", "a") as f:
        f.write(page)
end = "</head>  </html>"
with open("papers1.html", "a") as f:
    f.write(end)
for paper in papers:
    printmd("**[{}]({})**<br>{}<br><br>".format(paper["title"],
                                                paper["url"],
                                                paper["summary"]))

NameError: name 'date' is not defined

# this is a failure attempt. we cannot just send the code to the prompt, because each time it will create table of different sytle

In [ ]:
page = f"<html> <head> <h1>Daily Dose of AI Research</h1> <h4>{date.today()}</h4> <p><i>Summaries generated with: {LLM}</i>"
with open("papers.html", "w") as f:
    f.write(page)
for paper in papers:
    page = f'<h2><a href="{paper["url"]}">{paper["title"]}</a></h2> <p>{paper["summary"]}</p>'
    with open("papers.html", "a") as f:
        f.write(page)
end = "</head>  </html>"
with open("papers.html", "a") as f:
    f.write(end)
for paper in papers:
    printmd("**[{}]({})**<br>{}<br><br>".format(paper["title"],
                                                paper["url"],
                                                paper["summary"]))

**[VCode: a Multimodal Coding Benchmark with SVG as Symbolic Visual
  Representation](https://arxiv.org/pdf/2511.02778)**<br>```html
<html>
<head>
<h1>Daily Dose of AI Research</h1>
<h4>2023-10-27</h4>
<p><i>Summaries generated with: GPT-4</i>
</head>
<h2><a href="https://csu-jpg.github.io/VCode">VCode: a Multimodal Coding Benchmark with SVG as Symbolic Visual Representation</a></h2>
<p>The research article introduces VCode, a novel multimodal coding benchmark that redefines multimodal understanding as the generation of Scalable Vector Graphics (SVG) code from natural images. This approach emphasizes capturing symbolic visual meaning, similar to how humans reason over sketches, as an alternative to dense pixel-based representations. VCode repurposes existing benchmarks across commonsense, professional knowledge, and visual-centric perception domains. To evaluate symbolic fidelity, the authors propose CodeVQA, where a policy model answers questions based on the rendered SVG. Empirically, frontier Vision-Language Models (VLMs) demonstrate limitations in generating faithful SVGs. To address this, the paper presents VCoder, an agentic framework that augments VLMs with iterative revision (thinking with revision) and external perception tools (acting with visual tools). VCoder achieves significant performance gains over strong baselines, yet human studies and experimental results highlight persistent challenges in bridging the gap between language-centric and visual-centric coding, particularly in complex domains like professional knowledge and 3D reasoning.</p>
<table>
  <thead>
    <tr>
      <th>Strength</th>
      <th>Weakness</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>VCode introduces a novel and challenging perspective on multimodal understanding, framing it as SVG code generation to capture symbolic visual meaning, which moves beyond traditional pixel-based representations. The benchmark includes diverse domains (commonsense, professional, vision-centric) and a robust evaluation protocol (CodeVQA) specifically designed to assess symbolic fidelity. VCoder provides an effective solution by augmenting VLMs with iterative revision and external visual tools, significantly improving SVG generation accuracy and demonstrating a pathway to bridge the gap between language-centric and visual-centric coding. The research provides insights through human studies, suggesting alignment between human and VLM reasoning over symbolic representations, and the project is open-sourced, facilitating further research.</td>
      <td>Despite advancements, a substantial performance gap remains between the best SVG generation results and reasoning over original raw images, indicating that SVG may not yet fully capture all necessary visual information for complex tasks. The task of faithful SVG generation from natural images is highly challenging for frontier VLMs, requiring management of long code contexts and precise low-level detail encoding, which currently limits their intrinsic capabilities. VCoder's reliance on external visual tools (e.g., object detectors, segmenters) highlights that VLMs currently lack the intrinsic fine-grained perception needed for detailed code generation, potentially limiting end-to-end learning. Models show particular struggle with domains requiring professional knowledge and 3D reasoning, suggesting limitations in abstract or complex spatial representation via SVG. The observation that converting images to linguistic captions *before* SVG generation often outperforms direct image-to-SVG approaches implies that language still acts as a critical intermediate representation, rather than models directly "thinking" in visual code.</td>
    </tr>
  </tbody>
</table>
</html>
```<br><br>

**[When Visualizing is the First Step to Reasoning: MIRA, a Benchmark for
  Visual Chain-of-Thought](https://arxiv.org/pdf/2511.02779)**<br>```html
<html>
<head>
    <h1>Daily Dose of AI Research</h1>
    <h4>2025-11-05</h4>
    <p><i>Summaries generated with: MyLLM</i>
    <h2><a href="https://arxiv.org/abs/2511.02779">When Visualizing is the First Step to Reasoning: MIRA, a Benchmark for Visual Chain-of-Thought</a></h2>
    <p>The research introduces MIRA (Multimodal Imagination for Reasoning Assessment), a novel benchmark designed to evaluate Multimodal Large Language Models (MLLMs) in complex reasoning tasks where generating intermediate visual images is crucial, mimicking human "drawing to think." Comprising 546 multimodal problems across 20 task types, MIRA emphasizes scenarios requiring spatial relationships or structural reasoning difficult to articulate purely through language. The study employs a three-level evaluation protocol (direct, text-only CoT, and simulated Visual-CoT with annotated visual clues) and reveals that while leading MLLMs struggle significantly with direct inputs or text-based reasoning, their performance substantially improves when provided with intermediate visual cues, underscoring the vital role of visual information in advanced reasoning but also highlighting a current limitation in models' autonomous visual generation capabilities.</p>
    <table>
        <thead>
            <tr>
                <th>Strength</th>
                <th>Weakness</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>MIRA introduces a novel and challenging benchmark specifically designed to evaluate MLLMs on tasks requiring intermediate visual chain-of-thought (Visual-CoT) reasoning, filling a critical gap where existing benchmarks fall short.</td>
                <td>Current state-of-the-art MLLMs, including powerful closed-source models like GPT-5, demonstrate very poor performance on MIRA tasks with direct inputs or text-only CoT, highlighting a fundamental deficiency in their visual reasoning capabilities.</td>
            </tr>
            <tr>
                <td>The benchmark provides a high-quality dataset of 546 multimodal problems across 20 diverse task types, meticulously curated with human-annotated intermediate visual steps to enable rigorous evaluation of visual reasoning.</td>
                <td>The significant performance gains observed with "Simulated Visual-CoT" rely on human-provided intermediate images, indicating that current MLLMs cannot autonomously generate these crucial visual steps, thus limiting true "thinking while drawing" capabilities.</td>
            </tr>
            <tr>
                <td>A robust three-level diagnostic evaluation protocol (Direct, Text-CoT, and Simulated Visual-CoT) allows for a fine-grained analysis of models' abilities and the impact of visual cues, clearly distinguishing visual from textual reasoning.</td>
                <td>Text-only Chain-of-Thought (CoT) is largely ineffective or can even degrade performance for MIRA tasks, underscoring its inherent limitation for problems that are intrinsically visual and spatial.</td>
            </tr>
            <tr>
                <td>The study clearly demonstrates the substantial benefit of integrating intermediate visual information, showing an average relative performance gain of 33.7% across models when Visual-CoT cues are provided, affirming its potential.</td>
                <td>Open-weight MLLMs exhibit more limited performance gains even with Visual-CoT, suggesting architectural or training limitations that prevent them from fully leveraging visual clues.</td>
            </tr>
            <tr>
                <td>MIRA serves as a reproducible platform and metric system to drive the development of new MLLMs and training paradigms capable of integrating visual and textual reasoning for complex problem-solving.</td>
                <td>Attempts to probe the models' upper bounds (e.g., Pass@k, majority voting, specialized prompts) yield only modest improvements, further suggesting a deep-seated lack of core visual reasoning skill rather than just accidental errors.</td>
            </tr>
        </tbody>
    </table>
</head>
</html>
```<br><br>

**[When Modalities Conflict: How Unimodal Reasoning Uncertainty Governs
  Preference Dynamics in MLLMs](https://arxiv.org/pdf/2511.02243)**<br>This research introduces a novel framework for understanding how Multimodal Large Language Models (MLLMs) resolve conflicting information, decomposing "modality following" into case-specific relative reasoning uncertainty and a model's inherent modality preference. Through a carefully constructed controllable "toy" dataset that systematically varies visual and textual difficulty, the study empirically establishes a universal law: the probability of an MLLM following a specific modality monotonically decreases as its relative reasoning uncertainty increases. This allows for a principled quantification of a model's inherent preference as a "balance point," effectively disentangling it from unimodal capabilities and dataset artifacts, a significant strength over prior coarse dataset-level statistics. Furthermore, the paper provides mechanistic insight by revealing that models exhibit internal "oscillations" between conflicting answers across layers when operating in ambiguous regions near this balance point, explaining observed external hesitation. While the use of a controlled "toy" dataset is crucial for isolating variables and revealing these fundamental principles, its simplicity, focusing primarily on color and attribution tasks, might limit the direct generalizability of the specific balance points or oscillation patterns to more complex, real-world multimodal scenarios or diverse reasoning tasks. Additionally, the reliance on output token entropy as a proxy for perceived uncertainty, while validated, may not capture all nuances of reasoning difficulty across diverse MLLM behaviors.

```html
<table border="1">
  <thead>
    <tr>
      <th>Strength</th>
      <th>Weakness</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Proposes a novel framework: Decomposes modality following into relative reasoning uncertainty and inherent modality preference.</td>
      <td>Reliance on a "toy" dataset with simple tasks (color/attribution) may limit direct generalizability to complex, real-world multimodal reasoning scenarios.</td>
    </tr>
    <tr>
      <td>Introduces a controllable "toy" dataset: Systematically varies visual and textual difficulty for fine-grained analysis.</td>
      <td>Output token entropy, while validated as a proxy for perceived uncertainty, might not capture all nuances of reasoning difficulty across diverse MLLM behaviors.</td>
    </tr>
    <tr>
      <td>Discovers a universal monotonic law: Probability of following a modality monotonically decreases as its relative uncertainty increases.</td>
      <td></td>
    </tr>
    <tr>
      <td>Quantifies inherent modality preference: Uses a "balance point" to disentangle it from unimodal capabilities and dataset artifacts.</td>
      <td></td>
    </tr>
    <tr>
      <td>Provides mechanistic insight: Reveals internal "oscillations" between modalities in ambiguous regions, explaining external hesitation.</td>
      <td></td>
    </tr>
    <tr>
      <td>Offers a more principled understanding of MLLM decision dynamics by moving beyond coarse dataset-level statistics.</td>
      <td></td>
    </tr>
  </tbody>
</table>
```<br><br>

**[Brain-IT: Image Reconstruction from fMRI via Brain-Interaction
  Transformer](https://arxiv.org/pdf/2510.25976)**<br><table>
    <thead>
        <tr>
            <th>Strength</th>
            <th>Weakness</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>Achieves State-of-the-Art (SotA) results in fMRI-to-image reconstruction, demonstrating superior visual and quantitative faithfulness to seen images compared to prior methods.</td>
            <td>Reconstructions, while significantly improved, are not yet perfect, occasionally exhibiting inaccuracies in fine-grained details and semantics.</td>
        </tr>
        <tr>
            <td>Utilizes a brain-inspired design with a Brain Interaction Transformer (BIT) that effectively processes and integrates information from functionally similar brain-voxel clusters, which are shared across subjects.</td>
            <td>Potential limitations or residual inaccuracies in reconstructions may stem from the inherent resolution and complexity of the fMRI signal itself.</td>
        </tr>
        <tr>
            <td>Employs a novel dual-branch reconstruction pipeline, combining high-level semantic features (for diffusion model guidance) and low-level structural features (inverted via Deep Image Prior for initialization), ensuring comprehensive image recovery.</td>
            <td>Future research is needed to explore more expressive feature spaces that could address current method failures or enhance detail accuracy further.</td>
        </tr>
        <tr>
            <td>Demonstrates exceptional efficiency in transfer learning, enabling high-quality reconstructions from significantly limited subject-specific fMRI data (e.g., comparable performance to 40-hour methods with only 1 hour of data, and meaningful results from just 15 minutes).</td>
            <td></td>
        </tr>
    </tbody>
</table><br><br>

**[TWIST2: Scalable, Portable, and Holistic Humanoid Data Collection System](https://arxiv.org/pdf/2511.02832)**<br><table>
  <thead>
    <tr>
      <th>Strength</th>
      <th>Weakness</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>TWIST2 introduces a portable and MoCap-free humanoid teleoperation and data collection system, leveraging affordable consumer VR and a custom low-cost neck for essential egocentric stereo vision. This design facilitates full whole-body control, enabling the execution of long-horizon, dexterous, and mobile manipulation tasks. The system boasts highly scalable data collection capabilities, allowing a single operator to efficiently gather numerous high-quality demonstrations with low latency, and it underpins a novel hierarchical visuomotor policy learning framework for autonomous whole-body control. The entire system, data, and models are open-sourced, promoting reproducibility and further research.</td>
      <td>However, the system faces limitations, as its general motion tracker struggles with highly dynamic movements like sprinting, and the reliance on PICO's whole-body pose estimation, while portable, results in less accuracy compared to high-cost MoCap systems, particularly for untracked joints, which can diminish overall motion quality. Furthermore, the practical execution of continuous long-horizon tasks is occasionally constrained by underlying robot motor robustness issues (e.g., overheating), and the current autonomous policies, despite their innovative nature, exhibit limited generalization in some complex scenarios, such as performing only one-directional kicks.</td>
    </tr>
  </tbody>
</table><br><br>

...

**[iFlyBot-VLA Technical Report](https://arxiv.org/pdf/2511.01914)**<br><table>
    <thead>
        <tr>
            <th>Strength</th>
            <th>Weakness</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>
                iFlyBot-VLA introduces a powerful Vision-Language-Action (VLA) model for dual-arm robot control, achieving superior performance on the LIBERO benchmark (93.8% average accuracy) compared to existing state-of-the-art models like OpenVLA and π0. It demonstrates robust generalization capabilities to unseen objects, novel scenes, and varying illumination conditions in real-world pick-and-place tasks, maintaining high success rates. The model further excels in challenging, long-horizon, and dexterous manipulation tasks, such as parcel sorting and complex cloth folding, showcasing its reliability in fine-grained control. Its core strengths lie in a novel dual-level action representation framework—combining latent actions from large-scale videos with structured discrete action tokens—and a mixed training strategy that integrates robot trajectory data with general and spatial QA datasets, significantly enhancing the VLM's 3D perceptual and reasoning abilities. Additionally, the commitment to open-sourcing portions of their dataset and code will benefit the research community.
            </td>
            <td>
                Despite its strong performance, iFlyBot-VLA faces limitations typical of imitation learning approaches. It can struggle when encountering novel instructions involving entirely unseen concepts or objects, and similarly, it exhibits challenges in grasping objects with shapes it has never encountered before. The model's performance may degrade or fail to recover effectively when presented with out-of-distribution inputs during inference, indicating a need for future integration with reinforcement learning mechanisms to enhance its robustness and generalization beyond observed demonstrations.
            </td>
        </tr>
    </tbody>
</table><br><br>

**[D2D: Detector-to-Differentiable Critic for Improved Numeracy in
  Text-to-Image Generation](https://arxiv.org/pdf/2510.19278)**<br><html>
<head>
    <h1>Daily Dose of AI Research</h1>
    <h4>2024-06-03</h4>
    <p><i>Summaries generated with: GPT-4o</i></p>
    <h2><a href="https://arxiv.org/pdf/2510.19278v1">D2D: DETECTOR-TO-DIFFERENTIABLECRITIC FOR IMPROVEDNUMERACY INTEXT-TO-IMAGE GENERATION</a></h2>
    <p>
        <table>
            <thead>
                <tr>
                    <th>Strength</th>
                    <th>Weakness</th>
                </tr>
            </thead>
            <tbody>
                <tr>
                    <td>
                        The D2D (Detector-to-Differentiable) framework presents a novel solution to improve object numeracy in text-to-image (T2I) diffusion models, a common weakness where models struggle with generating the correct number of specified objects. Its core strength lies in transforming robust, yet previously non-differentiable, object detectors into differentiable critics using custom activation functions, allowing them to guide T2I models through initial noise optimization via a Latent Modifier Network. This approach consistently and substantially boosts object counting accuracy across diverse T2I architectures and benchmarks, correcting both under- and over-generations with improvements up to 13.7%, while maintaining image quality and incurring minimal computational overhead.
                    </td>
                    <td>
                        Despite these significant advancements, D2D's limitations include its continued struggle with high-density object scenarios and a lack of fine-grained control over object placement or attribute binding, as it is not a layout-control method. Additionally, as a T2I pipeline leveraging pre-trained diffusion models and detectors, it inherently carries the potential biases of these foundational components, necessitating caution in deployment.
                    </td>
                </tr>
            </tbody>
        </table>
    </p>
</head>
</html><br><br>

**[VidEmo: Affective-Tree Reasoning for Emotion-Centric Video Foundation
  Models](https://arxiv.org/pdf/2511.02712)**<br><table>
    <tr>
        <th>Strength</th>
        <th>Weakness</th>
    </tr>
    <tr>
        <td>VidEmo introduces a novel video foundation model for emotion understanding, integrating curriculum emotion learning and affective-tree reasoning to analyze facial attributes, expressions, and complex emotional states in a stage-wise, interpretable manner. A key strength is its state-of-the-art performance across 15 face perception tasks, significantly outperforming both open-source and proprietary VideoLLMs like Gemini 2.0, demonstrating superior fine-grained perception and rationale-driven emotional inference, supported by the new, large-scale, and meticulously curated Emo-CFG dataset.</td>
        <td>The model's limitations include its susceptibility to generating counterfactual content and its current reliance solely on visual input, lacking the integration of other crucial modalities like audio for truly comprehensive affective reasoning.</td>
    </tr>
</table><br><br>

**[LiveSecBench: A Dynamic and Culturally-Relevant AI Safety Benchmark for
  LLMs in Chinese Context](https://arxiv.org/pdf/2511.02366)**<br>```html
<table border="1">
    <thead>
        <tr>
            <th>Strength</th>
            <th>Weakness</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>LiveSecBench offers a critical, dynamic, and culturally-relevant AI safety benchmark specifically for Chinese-language Large Language Models (LLMs), addressing a significant gap where English-centric evaluations fall short on linguistic, cultural, and socio-political nuances. Its primary strengths include a continuous update mechanism that prevents model overfitting and keeps pace with evolving threats, a comprehensive six-dimensional evaluation framework (Legality, Ethics, Factuality, Privacy, Adversarial Robustness, and Reasoning Safety) deeply rooted in Chinese legal and social contexts, and a robust ELO rating system for fair and efficient model comparisons. The benchmark also demonstrates foresight by planning future expansions to cover Text-to-Image Generation Safety and Agentic Safety, while providing detailed, actionable reports to participating developers and maintaining dataset confidentiality to preserve integrity.</td>
            <td>Despite its strengths, LiveSecBench has a few notable weaknesses, primarily stemming from the non-public nature of its sensitive test dataset. While essential for preventing direct model training and maintaining integrity, this confidentiality limits transparency and independent reproducibility for external researchers who might wish to scrutinize the data or evaluation process. Furthermore, the dataset's construction, involving manual filtering, rewriting, and validation, can be resource-intensive, potentially affecting the agility or scale of its "dynamic" updates. The inherent subjectivity in defining and evaluating certain dimensions, such as Ethics and Legality, even within a specific cultural context, could lead to interpretational debates. Finally, the exclusion of "Reasoning Safety" from the overall average score calculation on the leaderboard might dilute the perceived holistic safety performance, and the computational demand of the ELO system could increase significantly with a growing number of models.</td>
        </tr>
    </tbody>
</table>
```<br><br>

**[Discriminately Treating Motion Components Evolves Joint Depth and
  Ego-Motion Learning](https://arxiv.org/pdf/2511.01502)**<br>DiMoDE is an unsupervised framework for joint depth and ego-motion learning that innovatively discriminates between motion components (rotation, tangential, radial translation) by analyzing their distinct rigid flows. Through optical axis and imaging plane alignments, it introduces explicit geometric constraints, reformulating learning into coaxial and coplanar forms to enhance robustness and stability for both depth and pose estimation.

<table>
  <thead>
    <tr>
      <th>Strength</th>
      <th>Weakness</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>
        DiMoDE achieves state-of-the-art performance in both visual odometry and depth estimation, particularly excelling under challenging conditions like varied illumination, adverse weather, and complex motions. It demonstrates strong accuracy and improved generalization on multiple public datasets and a newly collected real-world dataset. The framework also offers enhanced network convergence and training robustness while maintaining a lightweight architecture suitable for real-time applications.
      </td>
      <td>
        The method struggles to produce reliable predictions under extremely low illumination conditions due to severe degradation of photometric cues and contextual features. Additionally, its generalization capacity is limited for entirely unseen real-world environments with substantially higher complexity and diversity, requiring specific training data exposure for robust performance in such scenarios.
      </td>
    </tr>
  </tbody>
</table><br><br>

# This is the orginal output for the notebook file.

In [ ]:
for paper in papers:
    printmd("**[{}]({})**<br>{}<br><br>".format(paper["title"],
                                                paper["url"],
                                                paper["summary"]))

**[ThinkMorph: Emergent Properties in Multimodal Interleaved
  Chain-of-Thought Reasoning](https://arxiv.org/pdf/2510.27492)**<br>ThinkMorph proposes a unified model for multimodal reasoning that leverages complementary interleaved text and image thoughts, departing from isomorphic approaches. A significant strength of ThinkMorph is its ability to achieve substantial performance gains, averaging 34.7% over its base model on vision-centric benchmarks and generalizing effectively to out-of-domain tasks, even matching or surpassing larger and proprietary VLMs despite being fine-tuned on a relatively modest 24K high-quality reasoning traces. The research also highlights compelling emergent properties, including the model's capacity for unseen visual manipulations, autonomous switching between reasoning modes based on task complexity, and improved test-time scaling through diversified multimodal exploration. However, a weakness identified is that interleaved reasoning is not universally superior; for certain in-domain tasks like ChartQA, text-only reasoning slightly outperformed ThinkMorph, suggesting that visual input can sometimes be supplementary rather than essential. Additionally, test-time scaling on some perception-focused benchmarks exhibited a U-shaped pattern, initially declining before recovering, indicating that the benefits of diversified reasoning trajectories are not always consistently monotonic across all task types and might require larger sample sizes to fully manifest.<br><br>

**[OS-Sentinel: Towards Safety-Enhanced Mobile GUI Agents via Hybrid
  Validation in Realistic Workflows](https://arxiv.org/pdf/2510.24411)**<br>This research addresses the critical, underexplored challenge of ensuring safety in Vision-Language Model (VLM)-powered mobile GUI agents, which can pose risks like privacy leakage and system compromise despite their automation potential. The research introduces MobileRisk-Live, a dynamic Android sandbox for real-time safety studies, and MobileRisk, a corresponding benchmark of realistic, fine-grained annotated agent trajectories, providing a crucial foundation for reproducible research. Building on this, they propose OS-Sentinel, a novel hybrid safety detection framework that synergistically combines a Formal Verifier, which deterministically detects explicit system-level violations, such as file modifications or sensitive data patterns, and a VLM-based Contextual Judge that assesses nuanced contextual risks through semantic analysis of agent actions. Experiments demonstrate OS-Sentinel's superior performance, achieving 10-30% improvements over existing rule-based and VLM-as-a-Judge baselines across both step and trajectory-level detection, demonstrating its effectiveness for both real-time guarding and post-hoc analysis. However, the framework's Formal Verifier is dependent on system state traces, limiting its direct applicability to closed environments like iOS, and while the simulated and frozen environments show strong closeness, inherent discrepancies from real-world dynamic conditions, such as random push notifications, remain a potential limitation.<br><br>

**[INT v.s. FP: A Comprehensive Study of Fine-Grained Low-bit Quantization
  Formats](https://arxiv.org/pdf/2510.25602)**<br>This research systematically investigates the trade-offs between integer (INT) and floating-point (FP) quantization formats across varying granularities, addressing a gap in unified comparisons and challenging the industry's current FP-centric trajectory for AI hardware. A key strength is its revelation of a performance crossover, demonstrating that fine-grained MXINT8 formats consistently outperform MXFP8 in both algorithmic accuracy and hardware efficiency for 8-bit quantization, and proposing a symmetric clipping method for nearly lossless MXINT8 training. The study further provides a theoretical QSNR framework and empirical validation across a wide range of LLMs. However, a notable weakness is that INT formats do not universally outperform FP; for 4-bit quantization, FP often holds an accuracy advantage, and INT variants like NVINT4 only surpass their FP counterparts when combined with additional outlier-mitigation techniques like Hadamard rotation. Despite these nuances, the study powerfully advocates for prioritizing fine-grained INT formats in future AI accelerators due to their superior balance of accuracy, power, and efficiency.<br><br>

**[π_RL: Online RL Fine-tuning for Flow-based
  Vision-Language-Action Models](https://arxiv.org/pdf/2510.25889)**<br>This research introduces πRL, a novel open-source framework designed to enable online reinforcement learning (RL) fine-tuning for flow-based Vision-Language-Action (VLA) models like π0 and π0.5. Addressing the critical challenge of intractable action log-likelihoods inherent in their iterative denoising process, πRL proposes two distinct solutions: Flow-Noise, which models denoising as a discrete-time Markov Decision Process with a learnable noise network for exact log-likelihood computation, and Flow-SDE, which converts the ordinary differential equation denoising process into a stochastic differential equation, formulating a two-layer MDP with hybrid ODE-SDE sampling for efficiency. The framework demonstrates significant performance gains and enhanced generalization, boosting few-shot SFT models on benchmarks like LIBERO (e.g., π0.5 on LIBERO-Long from 43.9% to 94.0%) and achieving scalable multi-task RL across 4,352 combinations in ManiSkill, often surpassing full-dataset SFT baselines. However, the approach shows limited out-of-distribution generalization, particularly for semantic and action execution tasks, and the authors note potential for improvement in the noise injection strategy and the mixed ODE-SDE rollout for further training acceleration. A key limitation is the absence of real-world experimental validation.<br><br>

**[Continuous Autoregressive Language Models](https://arxiv.org/pdf/2510.27688)**<br>The research introduces Continuous Autoregressive Language Models (CALM), a novel paradigm aimed at overcoming the efficiency bottleneck of token-by-token LLM generation by shifting to continuous next-vector prediction. A key strength is its use of a high-fidelity, robust autoencoder to compress K tokens into a single continuous vector, reducing generative steps and significantly improving the performance-compute trade-off, with empirical results showing CALM achieving baseline performance at substantially lower FLOPs and establishing semantic bandwidth (K) as a powerful new scaling axis. To facilitate this, CALM develops a comprehensive likelihood-free toolkit, including an energy-based generative head for efficient single-step prediction, the BrierLM metric for principled evaluation (demonstrably correlating with perplexity), and algorithms for likelihood-free temperature sampling. However, the continuous domain presents inherent challenges: the autoencoder requires complex regularization to produce a robust latent space, and the specialized likelihood-free methods replace conventional, more direct approaches. CALM's initial performance at K=1 lags discrete baselines, indicating room for architectural optimization, while effectively leveraging higher K values appears to demand larger model capacities. Additionally, the provably exact temperature sampling algorithm is computationally intensive, necessitating approximations, and direct continuous vector input to the Transformer backbone proved suboptimal, requiring token reconstruction for input. The framework also exhibits a slower initial learning curve, suggesting a more complex learning task compared to discrete token prediction.<br><br>

...

**[Dual-Stream Diffusion for World-Model Augmented Vision-Language-Action
  Model](https://arxiv.org/pdf/2510.27607)**<br>DUST is a novel Vision-Language-Action (VLA) model that addresses the challenge of jointly predicting robotic actions and future visual observations by employing a Dual-Stream diffusion architecture. Its core strength lies in explicitly maintaining separate processing streams for action and vision tokens while enabling cross-modal knowledge sharing through a multimodal diffusion transformer, coupled with a decoupled training algorithm that applies independent noise and flow-matching losses to each modality. This design effectively handles their inherent statistical differences, allowing the model to learn bidirectional causal relationships and achieve consistent, significant performance gains (up to 6% in simulation, 13% in real-world tasks) over state-of-the-art baselines across diverse benchmarks, including strong transfer learning capabilities from action-free video. Additionally, DUST introduces an asynchronous joint sampling method for test-time scaling, further boosting performance by 2-5%. However, a potential weakness is that this asynchronous scaling for improved visual refinement comes at the cost of increased inference time, and while the world model predicts rich semantic embeddings, it does not directly generate pixel-level future observations, which might limit some applications requiring explicit visual synthesis.<br><br>

**[The Denario project: Deep knowledge AI agents for scientific discovery](https://arxiv.org/pdf/2510.26887)**<br>Paper not available<br><br>

**[Visual Backdoor Attacks on MLLM Embodied Decision Making via Contrastive
  Trigger Learning](https://arxiv.org/pdf/2510.27623)**<br>This research introduces BEAT, a novel framework for implanting object-based visual backdoor attacks into Multimodal Large Language Model (MLLM)-based embodied agents, causing them to execute attacker-specified multi-step policies when a visual trigger (an object in the environment) appears. A key strength of BEAT is its two-stage training scheme, combining supervised fine-tuning with a new Contrastive Trigger Learning (CTL) approach that uses preference learning to sharpen decision boundaries around triggers. This method enables high attack success rates (up to 80%) across various benchmarks and MLLMs, maintains strong or even improved benign task performance, and effectively generalizes to out-of-distribution trigger placements while achieving near-zero false activations. CTL is particularly noted for boosting activation accuracy by up to 39% and demonstrating data efficiency. However, a weakness is that the full effectiveness of CTL was not evaluated on proprietary MLLMs like GPT-4o due to current API limitations, and one of the benchmarks (VAB-OmniGibson) relies on bounding box annotations for trigger objects, which could simplify detection compared to real-world, unconstrained scenarios, though another benchmark (EB-ALFRED) was box-free. Additionally, attack failures in some cases were attributed to difficulties in detecting small or partially obstructed triggers and challenges in fine-grained navigation.<br><br>

**[Mask-to-Height: A YOLOv11-Based Architecture for Joint Building Instance
  Segmentation and Height Classification from Satellite Imagery](https://arxiv.org/pdf/2510.27224)**<br>This research introduces a YOLOv11-based architecture for joint building instance segmentation and discrete height classification from satellite imagery, capitalizing on YOLOv11's efficiency, multi-scale feature fusion, and real-time inference capabilities. A significant strength is its innovative reframing of height estimation as a robust, interpretable five-tier classification task, which enhances resilience to noisy Digital Surface Model (DSM) data and provides actionable outputs for urban planning, notably outperforming prior state-of-the-art continuous regression methods in both segmentation accuracy and inference speed on the DFC2023 dataset while effectively managing class imbalance. However, this focus on discrete height classification inherently sacrifices the granular detail offered by continuous regression, potentially limiting applications that require exact meter-level height values, and the method relies on a simplified mean-based height derivation from DSMs which might not fully capture complex building geometries.<br><br>

**[Limits of Generalization in RLVR: Two Case Studies in Mathematical
  Reasoning](https://arxiv.org/pdf/2510.27044)**<br>This research rigorously investigates Reinforcement Learning with Verifiable Rewards (RLVR) in fostering genuine mathematical reasoning, using two combinatorial problems—Activity Scheduling and Longest Increasing Subsequence—with carefully curated datasets featuring unique, verifiable optimal solutions. A key strength of the study is its controlled experimental design, employing various reward functions and precise evaluation metrics to measure not only answer accuracy but also reasoning fidelity. However, the findings reveal significant limitations of RLVR: while it often improves evaluation metrics, it frequently does so by reinforcing superficial heuristics, exploiting formatting strategies, or re-weighting existing solutions, rather than by acquiring new, genuine reasoning capabilities. For instance, LIS tasks saw a collapse in intermediate reasoning with answer-only rewards, and even when sequence rewards improved accuracy on Activity Scheduling, models demonstrated a disconnect, often emitting superficial "sorted" prefaces that did not reliably drive the final optimal schedule. Furthermore, specific reward designs, like directly rewarding sorting, could backfire and hinder learning. The study itself acknowledges a limitation in its focus on a single base model and two specific tasks, suggesting that the generalizability of these observations across more diverse models or problem domains warrants further investigation. Ultimately, the work emphasizes that RLVR can lead to apparent task generalization without strengthening underlying reasoning, highlighting the critical need for benchmarks that clearly differentiate genuine mathematical competence from shortcut exploitation.<br><br>